In [ ]:
pip install selenium webdriver-manager

In [3]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

# Kullanıcıdan yıl al
print("🗓️ Hangi yılın verilerini indirmek istiyorsunuz?")
print("📝 Mevcut yıllar: 2004-2025 arası")
year = input("➡️ Yıl girin (örn: 2023): ").strip()

# Yıl geçerliliğini kontrol et
try:
    year_int = int(year)
    if year_int < 2004 or year_int > 2025:
        print("❌ Hata: Yıl 2004-2025 arasında olmalıdır!")
        exit()
except ValueError:
    print("❌ Hata: Geçerli bir yıl girin!")
    exit()

print(f"✅ {year} yılı seçildi")

# Masaüstü yolunu al ve indirme konumunu ayarla
masaustu = os.path.join(os.path.expanduser("~"), "Desktop")
indir_konumu = os.path.join(masaustu, f"indirilen_exceller_{year}")
os.makedirs(indir_konumu, exist_ok=True)

print(f"📁 İndirme konumu: {indir_konumu}")

# Chrome seçeneklerini yapılandır
options = ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": indir_konumu,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,
    "safebrowsing.disable_download_protection": True,
    "plugins.always_open_pdf_externally": True
})

options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# WebDriver'ı başlat
driver_path = r"C:\chromedriver-win64\chromedriver.exe"
driver = webdriver.Chrome(service=ChromeService(driver_path), options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

wait = WebDriverWait(driver, 20)

try:
    print("🌐 Siteye bağlanılıyor...")
    driver.get("https://muhasebat.hmb.gov.tr/genel-butce-gelirlerinin-iller-itibariyle-tahakkuk-ve-tahsilati-2004-2019")
    time.sleep(3)
    
    # Sayfanın tamamen yüklenmesini bekle
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    print("✅ Sayfa yüklendi")
    
    print(f"\n{'='*50}")
    print(f"📅 {year} YILI İŞLENİYOR")
    print(f"{'='*50}")
    
    # ADIM 1: Yıl ana başlığını bul ve tıkla
    print(f"🔍 {year} yılı ana başlığı aranıyor...")
    year_main_found = False
    
    try:
        # Ana başlığı bul: "2025 Yılı Genel Bütçe Gelirlerinin İller İtibarıyla Tahakkuk ve Tahsilatı"
        year_main_elements = driver.find_elements(By.XPATH, f"//a[contains(text(), '{year} Yılı Genel Bütçe Gelirlerinin İller İtibarıyla Tahakkuk ve Tahsilatı')]")
        
        for element in year_main_elements:
            if element.is_displayed():
                print(f"🟢 {year} ana başlığı bulundu")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
                time.sleep(1)
                
                try:
                    element.click()
                except ElementClickInterceptedException:
                    driver.execute_script("arguments[0].click();", element)
                
                year_main_found = True
                time.sleep(2)
                break
                
    except Exception as e:
        print(f"Ana başlık arama hatası: {e}")
    
    if not year_main_found:
        print(f"❌ {year} ana başlığı bulunamadı!")
        print("💡 Lütfen yılın doğru olduğunu ve sayfada mevcut olduğunu kontrol edin.")
        exit()
    
    print(f"✅ {year} ana başlığı açıldı")
    
    # ADIM 2: "Bütçe Gelir Tabloları" alt başlığını bul ve tıkla
    print(f"🔍 {year} - Bütçe Gelir Tabloları alt başlığı aranıyor...")
    budget_tables_found = False
    
    try:
        # "Bütçe Gelir Tabloları" linkini bul
        budget_elements = driver.find_elements(By.XPATH, "//a[contains(text(), 'Bütçe Gelir Tabloları')]")
        
        for element in budget_elements:
            if element.is_displayed():
                print(f"🟢 Bütçe Gelir Tabloları alt başlığı bulundu")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
                time.sleep(1)
                
                try:
                    element.click()
                except ElementClickInterceptedException:
                    driver.execute_script("arguments[0].click();", element)
                
                budget_tables_found = True
                time.sleep(2)
                break
                
    except Exception as e:
        print(f"Alt başlık arama hatası: {e}")
    
    if not budget_tables_found:
        print(f"❌ {year} için Bütçe Gelir Tabloları bulunamadı!")
        print("💡 Bu yıl için veriler farklı bir yapıda olabilir.")
        exit()
    
    print(f"✅ {year} - Bütçe Gelir Tabloları açıldı")
    
    # ADIM 3: Bütçe Gelir Tabloları altındaki Excel dosyalarını bul ve indir
    print(f"🔍 {year} - Bütçe Gelir Tabloları altındaki Excel dosyaları aranıyor...")
    
    try:
        # Excel dosyalarını ara (farklı stratejilerle)
        excel_links = []
        
        # Strateji 1: .xlsx/.xls uzantılı linkler
        xlsx_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.xlsx') or contains(@href, '.xls')]")
        excel_links.extend(xlsx_links)
        
        # Strateji 2: Excel metni içeren linkler
        excel_text_links = driver.find_elements(By.XPATH, "//a[contains(text(), 'Excel') or contains(text(), 'excel')]")
        excel_links.extend(excel_text_links)
        
        # Strateji 3: İl isimlerini içeren Excel linkler
        il_excel_links = driver.find_elements(By.XPATH, "//a[contains(text(), 'Adana') or contains(text(), 'Ankara') or contains(text(), 'İstanbul') or contains(text(), 'Merkezi') or contains(text(), 'İl ')]")
        for link in il_excel_links:
            href = link.get_attribute('href')
            if href and ('.xlsx' in href or '.xls' in href):
                excel_links.append(link)
        
        # Tekrar eden linkleri temizle
        unique_links = []
        seen_hrefs = set()
        
        for link in excel_links:
            href = link.get_attribute('href')
            if href and href not in seen_hrefs and link.is_displayed():
                seen_hrefs.add(href)
                unique_links.append(link)
        
        excel_links = unique_links
        
        if excel_links:
            print(f"📊 {year} için {len(excel_links)} Excel dosyası bulundu")
            year_downloads = 0
            
            for idx, link in enumerate(excel_links, 1):
                try:
                    link_text = link.text.strip() if link.text else f"Excel_{year}_{idx}"
                    link_href = link.get_attribute('href')
                    
                    print(f"➡️ {year} - {idx}/{len(excel_links)} - {link_text}")
                    print(f"   URL: {link_href}")
                    
                    # Excel dosyasını indirmek için farklı yöntemler dene
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", link)
                    time.sleep(1)
                    
                    download_success = False
                    
                    # Yöntem 1: Direkt tıklama (Chrome genellikle bu yöntemle iyi çalışır)
                    try:
                        # Önce mevcut pencere sayısını al
                        current_windows = driver.window_handles
                        
                        # Chrome'da direkt tıklama genellikle çalışır
                        link.click()
                        time.sleep(2)
                        
                        # Yeni pencere açıldıysa kapat
                        new_windows = driver.window_handles
                        if len(new_windows) > len(current_windows):
                            driver.switch_to.window(new_windows[-1])
                            driver.close()
                            driver.switch_to.window(current_windows[0])
                        
                        print(f"✅ Direkt tıklama ile indirildi: {link_text}")
                        download_success = True
                        
                    except Exception as e:
                        print(f"🔄 Direkt tıklama başarısız, JavaScript deneniyor...")
                    
                    # Yöntem 2: JavaScript ile download attribute ekle
                    if not download_success:
                        try:
                            # Safe filename oluştur
                            safe_filename = "".join(c for c in link_text if c.isalnum() or c in (' ', '-', '_')).rstrip()
                            if not safe_filename.endswith(('.xlsx', '.xls')):
                                safe_filename += '.xlsx'
                            
                            download_script = f"""
                            var link = arguments[0];
                            var downloadLink = document.createElement('a');
                            downloadLink.href = link.href;
                            downloadLink.download = '{safe_filename}';
                            downloadLink.target = '_self';
                            document.body.appendChild(downloadLink);
                            downloadLink.click();
                            document.body.removeChild(downloadLink);
                            """
                            driver.execute_script(download_script, link)
                            time.sleep(2)
                            
                            print(f"✅ JavaScript ile indirildi: {link_text}")
                            download_success = True
                            
                        except Exception as e:
                            print(f"🔄 JavaScript başarısız, requests deneniyor...")
                    
                    # Yöntem 3: Requests ile direkt indirme
                    if not download_success:
                        try:
                            headers = {
                                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                                'Accept': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet,*/*',
                            }
                            
                            response = requests.get(link_href, headers=headers, timeout=30)
                            response.raise_for_status()
                            
                            safe_filename = "".join(c for c in link_text if c.isalnum() or c in (' ', '-', '_')).rstrip()
                            if not safe_filename.endswith(('.xlsx', '.xls')):
                                safe_filename += '.xlsx'
                            
                            file_path = os.path.join(indir_konumu, safe_filename)
                            with open(file_path, 'wb') as file:
                                file.write(response.content)
                            
                            print(f"✅ Requests ile indirildi: {link_text}")
                            download_success = True
                            
                        except Exception as e:
                            print(f"❌ Tüm yöntemler başarısız: {e}")
                    
                    if download_success:
                        year_downloads += 1
                    
                    # İndirme için bekle
                    time.sleep(1.5)
                    
                except Exception as e:
                    print(f"❌ {link_text if 'link_text' in locals() else 'Bilinmeyen dosya'} için hata: {e}")
                    continue
            
            print(f"\n{'='*60}")
            print(f"🎉 {year} YILI TAMAMLANDI!")
            print(f"📊 Toplam {year_downloads} dosya indirildi")
            print(f"📁 İndirme konumu: {indir_konumu}")
            print(f"{'='*60}")
            
        else:
            print(f"❌ {year} için Excel dosyası bulunamadı")
            print("💡 Bu yıl için veriler farklı bir formatta olabilir.")
            
    except Exception as e:
        print(f"❌ Excel arama hatası: {e}")
    
    # Tüm indirme işlemleri tamamlanana kadar bekle
    print("⏳ Tüm indirme işlemleri tamamlanana kadar bekleniyor...")
    time.sleep(5)

except TimeoutException:
    print("❌ Sayfa yükleme zaman aşımı")
except Exception as e:
    print(f"❌ Genel hata: {e}")
finally:
    print("🏁 Tarayıcı kapatılıyor...")
    time.sleep(3)
    driver.quit()
    print("✅ İşlem tamamlandı")
    
    # Sonuç özeti
    print(f"\n{'='*50}")
    print(f"📋 İŞLEM ÖZETİ")
    print(f"{'='*50}")
    print(f"🗓️ İşlenen Yıl: {year}")
    print(f"📁 İndirme Klasörü: {indir_konumu}")
    print(f"💾 Dosyalar masaüstünüzde 'indirilen_exceller_{year}' klasöründe")
    print(f"{'='*50}")

🗓️ Hangi yılın verilerini indirmek istiyorsunuz?
📝 Mevcut yıllar: 2004-2025 arası
➡️ Yıl girin (örn: 2023): 2022
✅ 2022 yılı seçildi
📁 İndirme konumu: C:\Users\HUSOCAN\Desktop\indirilen_exceller_2022
🌐 Siteye bağlanılıyor...
✅ Sayfa yüklendi

📅 2022 YILI İŞLENİYOR
🔍 2022 yılı ana başlığı aranıyor...
🟢 2022 ana başlığı bulundu
✅ 2022 ana başlığı açıldı
🔍 2022 - Bütçe Gelir Tabloları alt başlığı aranıyor...
🟢 Bütçe Gelir Tabloları alt başlığı bulundu
✅ 2022 - Bütçe Gelir Tabloları açıldı
🔍 2022 - Bütçe Gelir Tabloları altındaki Excel dosyaları aranıyor...
📊 2022 için 82 Excel dosyası bulundu
➡️ 2022 - 1/82 - 00-Merkez-2022
   URL: https://ms.hmb.gov.tr/uploads/sites/3/2024/03/00-Merkez-2022.xls
✅ Direkt tıklama ile indirildi: 00-Merkez-2022
➡️ 2022 - 2/82 - 01-Adana-2022
   URL: https://ms.hmb.gov.tr/uploads/sites/3/2024/03/01-Adana-2022.xls
✅ Direkt tıklama ile indirildi: 01-Adana-2022
➡️ 2022 - 3/82 - 02-Adıyaman-2022
   URL: https://ms.hmb.gov.tr/uploads/sites/3/2024/03/02-Adiyaman-20

✅ Direkt tıklama ile indirildi: 48-Muğla-2022
➡️ 2022 - 50/82 - 49-Muş-2022
   URL: https://ms.hmb.gov.tr/uploads/sites/3/2024/03/49-Mus-2022.xls
✅ Direkt tıklama ile indirildi: 49-Muş-2022
➡️ 2022 - 51/82 - 50-Nevşehir-2022
   URL: https://ms.hmb.gov.tr/uploads/sites/3/2024/03/50-Nevsehir-2022.xls
✅ Direkt tıklama ile indirildi: 50-Nevşehir-2022
➡️ 2022 - 52/82 - 51-Niğde-2022
   URL: https://ms.hmb.gov.tr/uploads/sites/3/2024/03/51-Nigde-2022.xls
✅ Direkt tıklama ile indirildi: 51-Niğde-2022
➡️ 2022 - 53/82 - 52-Ordu-2022
   URL: https://ms.hmb.gov.tr/uploads/sites/3/2024/03/52-Ordu-2022.xls
✅ Direkt tıklama ile indirildi: 52-Ordu-2022
➡️ 2022 - 54/82 - 53-Rize-2022
   URL: https://ms.hmb.gov.tr/uploads/sites/3/2024/03/53-Rize-2022.xls
✅ Direkt tıklama ile indirildi: 53-Rize-2022
➡️ 2022 - 55/82 - 54-Sakarya-2022
   URL: https://ms.hmb.gov.tr/uploads/sites/3/2024/03/54-Sakarya-2022.xls
✅ Direkt tıklama ile indirildi: 54-Sakarya-2022
➡️ 2022 - 56/82 - 55-Samsun-2022
   URL: https://ms